In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input,Conv2D,BatchNormalization,Concatenate,Conv2DTranspose,MaxPooling2D,Activation,Layer,UpSampling2D

2025-02-08 19:38:11.418101: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-08 19:38:11.429083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739023691.441368   12348 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739023691.444842   12348 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-08 19:38:11.456776: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [1]:
!pip show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/manik/tf-gpu/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: focal-loss


In [ ]:
from focal_loss import BinaryFocalLoss

In [4]:

class InPlaceABN(Layer):
    def __init__(self, activation='relu', momentum=0.99, epsilon=1e-3, **kwargs):

        super(InPlaceABN, self).__init__(**kwargs)
        self.momentum = momentum
        self.epsilon = epsilon
        self.activation = activation

    def build(self, input_shape):
        # Create trainable parameters for batch normalization
        self.gamma = self.add_weight(
            name='gamma', shape=(input_shape[-1],), initializer='ones', trainable=True
        )
        self.beta = self.add_weight(
            name='beta', shape=(input_shape[-1],), initializer='zeros', trainable=True
        )
        self.moving_mean = self.add_weight(
            name='moving_mean', shape=(input_shape[-1],), initializer='zeros', trainable=False
        )
        self.moving_variance = self.add_weight(
            name='moving_variance', shape=(input_shape[-1],), initializer='ones', trainable=False
        )
        super(InPlaceABN, self).build(input_shape)

    def call(self, inputs, training=None):
        # Compute batch statistics if in training mode
        if training:
            batch_mean, batch_var = tf.nn.moments(inputs, axes=[0, 1, 2], keepdims=False)
            # Update moving averages
            self.moving_mean.assign(self.moving_mean * self.momentum + batch_mean * (1 - self.momentum))
            self.moving_variance.assign(self.moving_variance * self.momentum + batch_var * (1 - self.momentum))
        else:
            # Use moving averages during inference
            batch_mean = self.moving_mean
            batch_var = self.moving_variance

        # Apply Batch Normalization
        normalized_inputs = (inputs - batch_mean) / tf.sqrt(batch_var + self.epsilon)
        bn_output = self.gamma * normalized_inputs + self.beta


        return tf.nn.relu(bn_output)


    def compute_output_shape(self, input_shape):
        return input_shape


In [5]:
# import tensorflow as tf

def dice_loss(y_true, y_pred, epsilon=1e-7):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])

    denominator = tf.reduce_sum(y_true + y_pred, axis=[1, 2, 3]) + epsilon

    dice_coefficient = numerator / denominator
    loss = 1 - dice_coefficient
    return tf.reduce_mean(loss)

fl=BinaryFocalLoss(gamma=3)

def fused_loss(y_true, y_pred, kappa=1.0):
    dice = dice_loss(y_true, y_pred)
    focal = fl(y_true, y_pred)
    total_loss = dice + kappa * focal
    return total_loss


In [ ]:
# def decoder_block(input,skip_features,num_filters):
#     x=Conv2DTranspose(num_filters,(2,2),strides=2,padding="same")(input)
#     x=Concatenate()([x,skip_features])   #jin do feature map ko combine kerna hai
#     x=conv_block(x,num_filters)
#     return x

# def conv_block(input,skip_features,num_filters):
#     x=Concatenate()([x,skip_features])
#     x=Conv2D(num_filters,3,padding="same")(input)
#     x=BatchNormalization()(x)
#     x=Activation("relu")(x)

#     x=Conv2D(num_filters,3,padding="same")(x)
#     x=BatchNormalization()(x)
#     x=Activation("relu")(x)

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Model

# def encoder(input_shape):
#     # Create the DenseNet121 model
#     base_model = tf.keras.applications.DenseNet121(weights=None, include_top=False, input_shape=input_shape)
#     base_model.trainable = True  # Allow training

#     # Define new inputs
#     inputs = tf.keras.Input(shape=input_shape)

#     # Pass inputs through the base DenseNet model
#     x = base_model(inputs)

#     # Extract intermediate outputs using the same computation graph
#     s1 = base_model.get_layer('conv1/relu').output  # First block output
#     dense_block1 = base_model.get_layer('pool2_relu').output  # First dense block
#     dense_block2 = base_model.get_layer('pool3_relu').output  # Second dense block
#     dense_block3 = base_model.get_layer('pool4_relu').output  # Third dense block
#     transition_block3 = base_model.get_layer('conv5_block1_0_relu').output  # Transition block output

#     # Return a functional model
#     return Model(inputs=inputs, outputs=[s1, dense_block1, dense_block2, dense_block3, transition_block3])

# # Initialize the encoder
# encoder_part = encoder((256, 256, 3))
# encoder_part.summary()


In [12]:
def encoder(input_shape):
    encoder_model = tf.keras.applications.DenseNet121(weights="imagenet", include_top=False, input_shape=input_shape)
    encoder_model.trainable =False

    inputs = tf.keras.Input(shape=input_shape)

    ouput= encoder_model(inputs)

    s1=encoder_model.get_layer('conv1_relu').output   #output for 4th conct operation
    s2=encoder_model.get_layer('pool1').output        #output for 3rd conct operation
    s3=encoder_model.get_layer('pool2_pool').output   #output for 2nd conct operation
    s4=encoder_model.get_layer('pool3_pool').output   #output for 1st conct operation
    s5=encoder_model.get_layer('pool4_pool').output   #ouput of encoder

    print(s1.shape)
    print(s2.shape)
    print(s3.shape)
    print(s4.shape)
    print(s5.shape)

    return Model(inputs=inputs, outputs=[s1,s2,s3,s4,s5],name="encoder")
    #return Model(inputs=inputs, outputs=transition_block3)

#pool2_relu->1st dense block end  (None, 128, 128, 256)
#pool3_relu  (None, 64, 64, 512)
#poo4_relu  (32,32,1024)
#relu

In [13]:
encoder_part=encoder((256,256,3))
s1,s2,s3,s4,s5=encoder_part.outputs   #num_filters_list=[256,512,256,64,32]

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
(None, 128, 128, 64)
(None, 64, 64, 64)
(None, 32, 32, 128)
(None, 16, 16, 256)
(None, 8, 8, 512)


In [ ]:
# encoder_part.summary()

In [14]:

def conv_block(input,num_filters):# yellow part in decoder

    x=Conv2D(num_filters,3,padding="same")(input)
    x=InPlaceABN(activation='relu')(x)
    x=Conv2D(num_filters,3,padding="same")(x)
    x=InPlaceABN(activation='relu')(x)
    return x

def decoder_block(input,skip_features,num_filters):# blue part in decoder

    x=Conv2DTranspose(num_filters,(2,2),strides=2,padding="same")(input)
    print(x.shape)
    x=Concatenate()([x,skip_features])
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = InPlaceABN(activation='relu')(x)
    return x



In [15]:
def decoder(input_shape, skip_connections, num_filters_list):
    inputs = Input(shape=input_shape)
    x = inputs
    s1,s2,s3,s4,s5=skip_connections

    x=decoder_block(x,s4,num_filters_list[0])
    x=conv_block(x,num_filters_list[0])

    x=decoder_block(x,s3,num_filters_list[1])
    x=conv_block(x,num_filters_list[1])

    x=decoder_block(x,s2,num_filters_list[2])
    x=conv_block(x,num_filters_list[2])

    x=decoder_block(x,s1,num_filters_list[3])
    x=conv_block(x,num_filters_list[3])

    # x=decoder_block(x,s1,num_filters_list[4])
    # x=conv_block(x,num_filters_list[4])

    x = Conv2D(filters=1, kernel_size=(1, 1), activation=None)(x)
    output = Activation('sigmoid')(x)


    return Model(inputs=inputs, outputs=output,name="decoder")




In [16]:

decoder_part = decoder(input_shape=(8,8,512), skip_connections=encoder_part.outputs, num_filters_list=[256,512,256,64,32])
# decoder_model.summary()

(None, 16, 16, 256)
(None, 32, 32, 512)
(None, 64, 64, 256)
(None, 128, 128, 64)


In [21]:
# encoder_part.summary()

In [22]:
# decoder_part.summary()

In [18]:
def DSUNET(encoder, decoder):
    inputs = encoder.input
    encoded = encoder(inputs)
    outputs = decoder(encoded)
    autoencoder = Model(inputs, outputs, name="DSUNET")
    return

In [ ]:
import numpy as np
from scipy.signal import convolve2d
import cv2

# Define SRM filters
SRM_Kernels = [
    (1/4) * np.array([
        [0, 0, 0, 0, 0],s
        [0, -1, 2, -1, 0],
        [0, 2, -4, 2, 0],
        [0, -1, 2, -1, 0],
        [0, 0, 0, 0, 0]
    ]),
    (1/12) * np.array([
        [-1, 2, -2, 2, -1],
        [2, -6, 8, -6, 2],
        [-2, 8, -12, 8, -2],
        [2, -6, 8, -6, 2],
        [-1, 2, -2, 2, -1]
    ]),
    (1/2) * np.array([
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 1, -2, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ])
]

def apply_srm_filter(image):
    # Initialize the output with the same shape as the input
    filtered_image = np.zeros((256, 256, 3), dtype=np.float32)

    # Apply each filter to each channel
    for i, kernel in enumerate(SRM_Kernels):
        for channel in range(3):  # Assuming 3 RGB channels
            filtered_image[..., i] += convolve2d(image[..., channel], kernel, mode='same', boundary='symm')

    return filtered_image

# # Load and preprocess a sample 256x256x3 image
# image = cv2.imread('/content/Tp_S_NRN_S_N_pla00005_pla00005_10937.jpg')  # Replace with your image path
# image = cv2.resize(image, (256, 256))  # Resize to 256x256
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# # Apply the SRM filter
# filtered_image = apply_srm_filter(image)
# print(image.shape)

# Display the result (normalize for visualization)
import matplotlib.pyplot as plt
# plt.imshow((filtered_image - filtered_image.min()) / (filtered_image.max() - filtered_image.min()))
# plt.title("Filtered Image with SRM")
# plt.show()


In [20]:
def ds_unet(input_shape):
    inputs = Input(shape=input_shape)

